In [19]:
import pandas as pd
import os
import glob
from pathlib import Path
#from dotenv import load_dotenv
import numpy as np
# import dask.dataframe as dd
# import dask.bag as db

import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
nat_img_dir='/nese/mit/group/sig/projects/naturalistic/nat_img/'

all_qc = pd.read_csv(  f"../data/all_qc.csv"  )
all_qc = all_qc[all_qc['task'].isin(['movieDM', 'movieTP'])]

def get_imaging_site(identifier):
    path = f'/nese/mit/group/sig/projects/hbn/hbn_bids/sub-{identifier}/'
    if os.path.isdir(path + 'ses-HBNsiteRU'):
        return 'HBNsiteRU'
    elif os.path.isdir(path + 'ses-HBNsiteCBIC'):
        return 'HBNsiteCBIC'
    elif os.path.isdir(path + 'ses-HBNsiteSI'):
        return 'HBNsiteSI'
    elif os.path.isdir(path + 'ses-HBNsiteCUNY'):
        return 'HBNsiteCUNY'
    else:
        return None
        
dx_final=pd.read_csv(f'{nat_img_dir}sourcedata/data/HBN/phenotype/parsed/dx_onehot.csv')
dx_final['imaging_site'] = dx_final['Identifiers'].apply(get_imaging_site)

filtered_df = dx_final[dx_final['Autism Spectrum Disorder'] == 1]
asd_df = filtered_df[['Identifiers','imaging_site']].rename(columns={'Identifiers': 'subj_id'})
asd_df['subj_id'] = 'sub-' + asd_df['subj_id'].astype(str)

filtered_df = dx_final[dx_final['No Diagnosis Given'] == 1]
nt_df = filtered_df[['Identifiers','imaging_site']].rename(columns={'Identifiers': 'subj_id'})
nt_df['subj_id'] = 'sub-' + nt_df['subj_id'].astype(str)

filtered_df = dx_final[(dx_final['No Diagnosis Given'] == 0) & (dx_final['Autism Spectrum Disorder'] == 0)]
pilot_df = filtered_df[['Identifiers','imaging_site']].rename(columns={'Identifiers': 'subj_id'})
pilot_df['subj_id'] = 'sub-' + pilot_df['subj_id'].astype(str)


dx_final

,Identifiers,Age,Sex,Site,ADHD-Combined Type,ADHD-Hyperactive/Impulsive Type,ADHD-Inattentive Type,Acute Stress Disorder,Adjustment Disorders,Agoraphobia,...,Unspecified Anxiety Disorder,Unspecified Attention-Deficit/Hyperactivity Disorder,Unspecified Bipolar and Related Disorder,Unspecified Depressive Disorder,Unspecified Feeding or Eating Disorder,Unspecified Intellectual Disability,Unspecified Neurodevelopmental Disorder,Unspecified Tic Disorder,Unspecified Trauma- and Stressor-Related Disorder,imaging_site
0,NDARAA075AMK,6.728040,female,2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HBNsiteSI
1,NDARAA112DMH,5.545744,male,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HBNsiteRU
2,NDARAA117NEJ,7.475929,male,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HBNsiteRU
3,NDARAA306NT2,21.216746,female,3,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HBNsiteRU
4,NDARAA358BPN,11.853296,male,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4762,NDARZT957CWG,16.034565,male,3,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HBNsiteCBIC
4763,NDARZU401RCU,17.663586,female,3,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HBNsiteCBIC
4764,NDARZV458LDP,15.763175,female,4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HBNsiteCUNY
4765,NDARZX745YLZ,16.288501,male,4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None


In [27]:
merged_qc = all_qc.merge(dx_final[['Identifiers']], left_on='sub', right_on='Identifiers', how='inner')
merged_qc

,aor,aqi,bids_meta,dummy_trs,dvars_nstd,dvars_std,dvars_vstd,efc,fber,fd_mean,...,summary_fg_stdv,tsnr,subj_id,run_bad,ses,sub,task,relaxed_fd50,dorit_fd40,Identifiers
0,0.001634,0.011370,"{'AcquisitionMatrixPE': 84, 'BandwidthPerPixel...",0,31.320980,0.998533,0.988834,0.489231,1225.898438,0.149726,...,114.659175,31.901540,sub-NDARAA306NT2,movieDM,HBNsiteRU,NDARAA306NT2,movieDM,1,1,NDARAA306NT2
1,0.030871,0.069020,"{'AcquisitionMatrixPE': 84, 'BandwidthPerPixel...",0,100.102069,1.191695,1.183819,0.537480,677.013855,2.853728,...,543.516007,11.624881,sub-NDARAA947ZG5,movieDM,HBNsiteCBIC,NDARAA947ZG5,movieDM,0,0,NDARAA947ZG5
2,0.000461,0.004402,"{'AcquisitionMatrixPE': 84, 'BandwidthPerPixel...",2,26.360379,1.038065,0.995616,0.480758,2013.960938,0.120238,...,120.056921,46.489295,sub-NDARAA948VFH,movieDM,HBNsiteRU,NDARAA948VFH,movieDM,1,1,NDARAA948VFH
3,0.014249,0.051829,"{'AcquisitionMatrixPE': 84, 'BandwidthPerPixel...",4,62.831196,1.060205,1.054771,0.510687,1170.000854,1.230934,...,81.522293,14.618530,sub-NDARAB055BPR,movieDM,HBNsiteRU,NDARAB055BPR,movieDM,0,0,NDARAB055BPR
4,0.001283,0.010898,"{'AcquisitionMatrixPE': 84, 'BandwidthPerPixel...",0,48.614155,1.172940,0.991999,0.535286,908.762024,0.342414,...,1370.364640,29.789868,sub-NDARAB282FDJ,movieDM,HBNsiteCUNY,NDARAB282FDJ,movieDM,0,0,NDARAB282FDJ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4259,0.006022,0.011833,"{'AcquisitionMatrixPE': 84, 'BandwidthPerPixel...",2,45.952413,1.085489,1.022024,0.522597,950.524414,0.458444,...,1421.131569,27.747678,sub-NDARZZ046HJA,movieTP,HBNsiteCUNY,NDARZZ046HJA,movieTP,0,0,NDARZZ046HJA
4260,0.001287,0.017197,"{'AcquisitionMatrixPE': 84, 'BandwidthPerPixel...",7,39.762260,1.032784,0.952145,0.522625,915.985474,0.216253,...,611.651143,25.922106,sub-NDARZZ598MH8,movieTP,HBNsiteCBIC,NDARZZ598MH8,movieTP,0,0,NDARZZ598MH8
4261,0.009926,0.018417,"{'AcquisitionMatrixPE': 84, 'BandwidthPerPixel...",0,44.143766,1.069409,1.053060,0.494117,1584.325195,0.541157,...,76.732486,26.877612,sub-NDARZZ740MLM,movieTP,HBNsiteRU,NDARZZ740MLM,movieTP,0,0,NDARZZ740MLM
4262,0.002157,0.013886,"{'AcquisitionMatrixPE': 84, 'BandwidthPerPixel...",4,42.032762,1.109099,1.003362,0.523672,1029.216553,0.230321,...,662.999336,29.904263,sub-NDARZZ741VJE,movieTP,HBNsiteCBIC,NDARZZ741VJE,movieTP,1,0,NDARZZ741VJE


In [ ]:
# get the max FD for each one


    confounds_file=f'/nese/mit/group/sig/projects/hbn/hbn_bids/derivatives/fmriprep_23.2.0/sub-{sub}/ses-HBNsiteRU/func/sub-{sub}_ses-HBNsiteRU_task-movieDM_desc-confounds_timeseries.tsv'
    df = pd.read_csv(confounds_file, sep='\t')
    metrics['Max_FD'].append(np.max(df['framewise_displacement']))

In [ ]:
import pandas as pd

# Initialize an empty list to store the max FD values
max_fd_values = []

for index, row in all_qc.iterrows():
    # Generate the confounds_file path based on the row's sub, ses, and task columns
    confounds_file = f'/nese/mit/group/sig/projects/hbn/hbn_bids/derivatives/fmriprep_23.2.0/sub-{row["sub"]}/ses-{row["ses"]}/func/sub-{row["sub"]}_ses-{row["ses"]}_task-{row["task"]}_desc-confounds_timeseries.tsv'
    
    # Load the confounds file and extract the max FD value
    confounds_df = pd.read_csv(confounds_file, sep='\t')
    max_fd_value = confounds_df['framewise_displacement'].max()
    
    # Append the max FD value to the list
    max_fd_values.append(max_fd_value)

# Assign the list as a new column in the DataFrame
all_qc['max_fd'] = max_fd_values
